<a href="https://colab.research.google.com/github/Nileshsri2022/genai/blob/main/Doc_summarizer_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install llama-index llama-index-llms-groq llama-index-embeddings-huggingface

In [3]:
import os

In [4]:
GROQ_API_KEY = "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"

In [5]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import SummaryIndex
from llama_index.llms.groq import Groq
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [9]:
llm = Groq(model="llama-3.1-8b-instant")

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# load the document
documents = SimpleDirectoryReader(input_files=["data/attention_is_all_you_need.pdf"]).load_data()

In [11]:
print(type(documents))

<class 'list'>


In [12]:
len(documents)

15

In [14]:
#documents[0]

In [16]:
#documents[1]

In [17]:
splitter = SentenceSplitter(chunk_size=2048)
nodes = splitter.get_nodes_from_documents(documents)

In [18]:
summary_index = SummaryIndex(nodes)

In [19]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)

In [20]:
response = summary_query_engine.query("Summarize the given document")

In [21]:
print(type(response))

<class 'llama_index.core.base.response.schema.Response'>


In [22]:
print(response)

A new approach to sequence transduction models has been proposed, which relies entirely on attention mechanisms to draw global dependencies between input and output sequences. This approach allows for significantly more parallelization and can reach a new state of the art in translation quality. The model, called the Transformer, consists of an encoder and a decoder, both of which are composed of identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. The Transformer achieves better results in terms of translation quality and parallelization compared to other models, including recurrent and convolutional neural networks. It also has the ability to learn dependencies between distant positions and its parallelization capabilities. The document presents several experiments on machine translation tasks, showing that the Transformer model outperforms previous state-of-the-art models with signifi

In [23]:
response = summary_query_engine.query("Give a detailed summary of teh given document")

print(response)

The document discusses a novel approach to sequence transduction models, which are used for tasks such as machine translation and language modeling. The current dominant models are based on complex recurrent or convolutional neural networks, but these models have limitations, including sequential computation and difficulty in learning dependencies between distant positions.

To address these limitations, the authors propose a new model architecture called the Transformer, which relies entirely on self-attention mechanisms to draw global dependencies between input and output sequences. This approach allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for a short period of time.

The Transformer architecture consists of an encoder and a decoder, both of which are composed of identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forwar

In [24]:
# storing the document summary index
summary_index.storage_context.persist("doc_summary_index")